### imports

In [ ]:

from pathlib import Path
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import CohereEmbeddings


import pinecone
from pinecone import Pinecone
from langchain.vectorstores import Pinecone as PineconeVectorStore

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA


from langchain.prompts import PromptTemplate

import os
from pathlib import Path
import gradio as gr

### config

In [107]:

PDF_DIR = Path("pdfs")
CHUNK_SIZE = 200
CHUNK_OVERLAP = 10


In [ ]:
from dotenv import load_dotenv
load_dotenv()  


openai_api_key = os.environ["OPENAI_API_KEY"]
cohere_api_key = os.environ["COHERE_API_KEY"]
pinecone_api_key = os.environ["PINECONE_API_KEY"]


### Document Loaders


In [116]:

def load_pdfs(pdf_dir) :
    texts = []
    for pdf_file in pdf_dir.glob("*.pdf"):
        reader = PdfReader(str(pdf_file))
        text = "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
        texts.append(text)
    return texts

texts = load_pdfs(PDF_DIR)
len(texts)


5

### split to chunks

In [117]:

def split_texts(texts) :
    splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    chunks = []
    for text in texts:
        chunks.extend(splitter.split_text(text))
    return chunks

chunks = split_texts(texts)   
len(chunks)

224

### Embeddings with cohere

In [118]:


embeddings = CohereEmbeddings(model="embed-multilingual-v3.0",cohere_api_key=cohere_api_key,  user_agent="langchain")
vectors = embeddings.embed_documents(chunks)


### Vector Store with Pinecone

In [119]:

pc = Pinecone(api_key=pinecone_api_key)
pinecone_index = pc.Index("my-rag-index")

to_upsert = [
    (f"chunk-{i}", vec, {"text": chunks[i]})
    for i, vec in enumerate(vectors)
]
pinecone_index.upsert(vectors=to_upsert)

{'upserted_count': 224}


### RAG

In [ ]:

def run_rag_pipeline(query: str):
    
    query_embedding = embeddings.embed_query(query)
    #Retrieval with Pinecone Query
    response = pinecone_index.query(
    vector=query_embedding, 
    top_k=4,                
    include_metadata=True   
    )


    for i, match in enumerate(response['matches']):
        print(f"\n-- קטע {i+1} --")
        print(match['metadata']['text'])

    #Generation with OpenAI
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        openai_api_key=openai_api_key,
        temperature=1.0
        )

    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template="""
        אתה תזונאי ודיאטן מומחה
        ענה על השאלה הבאה בהתבסס על ההקשר הנתון .
        אם אין מידע רלוונטי, אמור \"לא מצאתי מידע\".
        הקשר:
        {context}

        שאלה:
        {question}
        """
            )
    context = "\n\n".join([match['metadata']['text'] for match in response['matches']])
    prompt = prompt_template.format(context=context, question=query)

    answer = llm.invoke(prompt)
    return answer.content



In [121]:
run_rag_pipeline("איך מרפאים סוכרת?")


-- קטע 1 --
בשביל להחלים מסוכרת אין צורך לסבול. כמי שמלווה מאות רבות של סוכרתיים אני יכול לספר שחלקם  
הגדול נהנה הרבה יותר מהתפריט החדש והמגוון. ההחלמה מהסוכרת צריכה להיעשות בהחלט על ידי

-- קטע 2 --
באמצעות מספר עקרונות פשוטים אפשר לשפר באופן דרמטי את מצב הסוכרת, 
ובחלק גדול מהמקרים להגיע לריפוי מוחלט.   
מדי יום אני פוגש סוכרתיים חדשים שמקבלים חיים חדשים במתנה.

-- קטע 3 --
נזכיר בקצרה כי הסיבה המרכזית היא שקיעה של פסולת על הקולטנים, ובעיקר פסולת שומנית.   
הריפוי האמיתי למחלת הסוכרת הוא פינוי הפסולת ששקעה על הקולטנים, והחזרת הרגישות שלהם  
לאינסולין.

-- קטע 4 --
פרק5   
כך מרפאים סוכרת 
הכותרת נשמעת לכם יומרנית מדי?   
האם אכן יש דרך לרפא סוכרת?  
האם סוכרת היא לא מחלה כרונית?


'על פי הטקסט, ריפוי סוכרת מתבצע באמצעות פינוי פסולת (בעיקר שומנית) ששקעה על הקולטנים והחזרת הרגישות שלהם לאינסולין.'

### Gradio

In [ ]:
import gradio as gr

def build_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🔍 שאילתות על קבצי תזונה (RAG)")
        question = gr.Textbox(label="שאלה", placeholder="מה כתוב על ברזל בתפריט?")
        answer = gr.Textbox(label="תשובה")
        submit_btn = gr.Button("שאל")
        submit_btn.click(fn=run_rag_pipeline, inputs=question, outputs=answer)
    return demo

demo = build_interface()
demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.



-- קטע 1 --
בשביל להחלים מסוכרת אין צורך לסבול. כמי שמלווה מאות רבות של סוכרתיים אני יכול לספר שחלקם  
הגדול נהנה הרבה יותר מהתפריט החדש והמגוון. ההחלמה מהסוכרת צריכה להיעשות בהחלט על ידי

-- קטע 2 --
באמצעות מספר עקרונות פשוטים אפשר לשפר באופן דרמטי את מצב הסוכרת, 
ובחלק גדול מהמקרים להגיע לריפוי מוחלט.   
מדי יום אני פוגש סוכרתיים חדשים שמקבלים חיים חדשים במתנה.

-- קטע 3 --
בפגישה  הסברתי  לרבי  מיכאל  שהמקור  לסוכרת ה וא  'תנגודת של הגוף  לאינסולין ', וכי הדרך  לשפר   
את מצב  הסוכרת,  ואולי אפילו לרפא אותה,  הי א באמצעות  הסרת התנגודת. וכאשר  מצליחים לעשות

-- קטע 4 --
נזכיר בקצרה כי הסיבה המרכזית היא שקיעה של פסולת על הקולטנים, ובעיקר פסולת שומנית.   
הריפוי האמיתי למחלת הסוכרת הוא פינוי הפסולת ששקעה על הקולטנים, והחזרת הרגישות שלהם  
לאינסולין.
